## Градиентный бустинг на решающих деревьях

## <font color='blue'>XGBoost</font> eXtreme Gradient Boosting

XGBoost строит композицию из $K$ базовых алгоритмов $b_k$:

$$ \hat{y}_i = \hat{y}_i^{K} = \sum_{k=1}^{K} b_k(x_i) = \hat{y}_i^{\left(K - 1\right)} + b_K(x_i), $$

минимизируя следующий функционал:

$$ Obj = \sum_{i=1}^N \mathcal{L}(y_i, \hat{y}_i ) + \sum_{k=1}^{K} \Omega(b_k),$$

где
 - $N$ — размер обучающей выборки;
 - $x_i, y_i, \hat{y}_i$ — i-ый объект, правильный ответ и предсказание модели для него;
 - $\hat{y}_i^{t}$ — предсказание композиции из $t$ уже обученных базовых алгоритмов для i-го объекта;
 - $\Omega$ — регуляризатор;
 - $\mathcal{L}(y_i, \hat{y}_i)$ — функция потерь.


Функционал, оптимизируемый на $t$-ой итерации:

$$ Obj^{(t)} = \sum_{i=1}^N \mathcal{L}\left(y_i, \hat{y}_i^{(t-1)} + b_t(x_i)\right) + \Omega(b_t).$$

В случае бустинга над решающими деревьями регуляризатор имеет следующий вид:

$$ \Omega(b_t) = \gamma T + \frac{1}{2}\lambda\sum_{j=1}^{T}w_j^2 + \alpha\sum_{j=1}^{T}w_j,$$

где 
 - $T$ — количество листьев в дереве;
 - $w_j$ — веса в листьях дерева;
 - $\lambda, \alpha, \gamma$ — гиперпараметры.

###  Немного теории


Данный регуляризатор подобран эвристически и хорошо показывает себя на практике. 

Раскладывая в ряд Тейлора выражение $\mathcal{L}\left(y_i, \hat{y}_i^{\left(t-1\right)} + b_t(x_i)\right)$ до второго порядка, получаем:

$$ Obj^{(t)} = \sum_{i=1}^N\left[\mathcal{L}(y_i, \hat{y}_i^{\left(t-1\right)}) + g_{i}b_{t}(x_i) + \frac{1}{2}h_{i}b_{t}^2(x_i)\right] + \Omega(b_t),$$

где $g_i = \partial_{\hat{y}_i^{(t-1)}} \mathcal{L}(y_i, \hat{y_i}^{(t-1)})$, $h_i = \partial_{\hat{y}_i^{(t-1)}}^2 \mathcal{L}(y_i, \hat{y}_i^{(t-1)}) $ — градиент и гессиан оптимизируемой функции потерь.

Приводя теперь подобные слагаемые и отбрасывая слагаемое $ \mathcal{L}(y_i, \hat{y}_i^{(t-1)}) $, не зависящее от $ b_t(x_i)$ (а следовательно, не влияющее на точку минимума функционала), получаем формулу:
$$ Obj^{(t)} \simeq \sum_{j=1}^{T}\left[\sum_{i \in I_j} g_i w_j + \frac{1}{2}\sum_{i \in I_j} (h_i + \lambda)w_j^2\right] + \gamma T$$
$$ = \sum_{j=1}^{T}\left[G_jw_j + \frac{1}{2}(H_j + \lambda)w_j^2\right] + \gamma T, $$

где 
 - $ I_j $ - множество объектов обучающей выборки, попавших в $j$-ый лист дерева;
 - $ G_j = \sum_{i \in I_j} g_i$;
 - $ H_j = \sum_{i \in I_j} h_i$.
 
Теперь, имея заданную структуру дерева, можно аналитически вычислить оптимальные значения для весов:
$$ w_j^* = -\frac{G_j}{H_j + \lambda}.$$

Значение функционала при этом будет равно:

$$ Obj = -\frac{1}{2}\sum_{j=1}^T \frac{G_j^2}{H_j + \lambda} + \gamma T .$$

Осталось только построить дерево оптимальной структуры. Это можно делать известными методами построения решающих деревьев, проводя разбиения таким образом, чтобы максимизировать gain, определенный как уменьшение $Obj$ в момент этого разбиения. Для уже построенного дерева по формулам $ w_j^* $ вычисляются оптимальные значения в листьях.

### https://github.com/dmlc/xgboost

[Установка](https://xgboost.readthedocs.io/en/latest/build.html)

## Пример

Возьмем датасет Boston Housing и обучим XGBoost на нем.

In [95]:
import xgboost as xgb
from xgboost import XGBClassifier, XGBRegressor

import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix, mean_squared_error


rng = np.random.RandomState(31337)

boston = load_boston()
y = boston['target']
X = boston['data']

kf = KFold(n_splits=10)
import pandas as pd
pd.DataFrame(X)


,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
5,0.02985,0.0,2.18,0.0,0.458,6.430,58.7,6.0622,3.0,222.0,18.7,394.12,5.21
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0,311.0,15.2,395.60,12.43
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0,311.0,15.2,396.90,19.15
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0,311.0,15.2,386.63,29.93
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0,311.0,15.2,386.71,17.10


#### Преимущества:


* потенциально очень высокое качество во многих задачах

* находит нелинейные связи

* способен обработать датасеты с большим числом объектов и признаков

#### Недостатки:


* очень много параметров

* модели не интерпретируемы

* по умолчанию не очень быстрый

XGBoost предлагает 2 способа использования алгоритмов:
* sklearn-совместимые классы XGBClassifier, XGBRegressor

* "оригинальная" python-библиотека

###  sklearn

In [61]:
for fold_index, (train_index, test_index) in enumerate(kf.split(X)):
    xgb_model = xgb.XGBRegressor().fit(X[train_index], y[train_index])
    predictions = xgb_model.predict(X[test_index])
    actuals = y[test_index]
    print ("RMSE on fold {}: {}".format(fold_index, np.sqrt(mean_squared_error(actuals, predictions))))

RMSE on fold 0: 3.126266920192809
RMSE on fold 1: 2.2050842142515132
RMSE on fold 2: 3.4390470416794554
RMSE on fold 3: 4.166562811870727
RMSE on fold 4: 3.5296880233146
RMSE on fold 5: 3.7556842429604136
RMSE on fold 6: 3.003147478207282
RMSE on fold 7: 8.977155169294026
RMSE on fold 8: 4.259501252709266
RMSE on fold 9: 3.7372037090240595


### xgboost python

In [96]:
def get_params():
    params = {}
    params["objective"] = "reg:linear"
    params["booster"] = "gbtree"
    params["eval_metric"] = "rmse"
    params["num_boost_round"] = 100
    params["max_depth"] = 3
    params["tree_method"] = "approx"
    params["sketch_eps"] = 1
    
    return params
    
for fold_index, (train_index, test_index) in enumerate(kf.split(X)):

    params = get_params()
    
    xgtrain = xgb.DMatrix(X[train_index], label=y[train_index])
    xgtest = xgb.DMatrix(X[test_index], label=y[test_index])

    bst = xgb.train(params, xgtrain)

    print ("RMSE on fold {}: {}".format(fold_index, bst.eval(xgtest)))

[19:01:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[19:01:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[19:01:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:01:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:01:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:01:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:01:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:01:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:01:08] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_

## Особенности XGBoost

<font size=3>
Написан на C++, есть обертки на Python, R, Java, Scala

С помощью XGBoost выиграна половина конкурсов на Kaggle

Существует коммерческая версия TreeNet

### Регуляризация

<font size=3>
Для уменьшения переобучения целевая функция поддерживает L0, L1, L2 регуляризации

###  Параллелизм (по признакам)

<img src="http://zhanpengfang.github.io/fig_418/feature_speedup.png" height="600" width="600">

<font size=3>
Также есть возможность запускаться на Hadoop, Spark, Flink и DataFlow

### Кастомные функции потерь / метрики качества

В XGBoost встроено множество различных функций потерь:

* reg:linear

* reg:logistic

* binary:logistic

* binary:logitraw

* multi:softmax

* rank:pairwise

* ...

А также соответствующих eval_metric, которые замеряют качество и позволяют сделать early stop.

Но также имеется возможность реализовать свой objective и eval_metric.

Все, что для этого нужно - уметь считать градиент и гессиан.

In [97]:
def my_reg_linear(preds, dtrain):
    labels = dtrain.get_label()
    grad = (preds - labels)
    hess = np.ones(labels.shape[0])
    return grad, hess

In [101]:
for train_index, test_index in kf.split(X):
    params = get_params()
    
    xgtrain = xgb.DMatrix(X[train_index], label=y[train_index])
    xgtest = xgb.DMatrix(X[test_index], label=y[test_index])

    bst = xgb.train(params, xgtrain, obj=my_reg_linear)
    
    predictions = bst.predict(xgtest)
    actuals = y[test_index]

    print (bst.eval(xgtest))

[19:06:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[19:06:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 10 extra nodes, 0 pruned nodes, max_depth=3
[19:06:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:06:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 14 extra nodes, 0 pruned nodes, max_depth=3
[19:06:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:06:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:06:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:06:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_depth=3
[19:06:14] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 12 extra nodes, 0 pruned nodes, max_

### Approximated tree splitting

Если данных слишком много, то можно использовать не все значения признаков, а разделить их на бакеты.

А именно, от каждого признака берутся не все значения, а только некоторое подмножество. Разбиение производится по элементам этого подмножества. 

Для разбиения выбираются взвешенные перцентили.

В оригинальной статье указывается 2 алгоритма:
*   глобальный - один раз выбрать разбиение значений фактора перед началом построения дерева и зафиксировать

    экономим на выборе разбиений, но обычно приходится выбирать больше точек разбиения
    
    
*   локальный - выбирать разбиение после каждого сплита
  
    работает лучше на глубоких деревьях

In [ ]:
params["tree_method"] = "approx"
params["sketch_eps"] = 0.2

### Пропуски в данных

XGBoost умеет обрабатывать разреженные матрицы

Но категориальные признаки нужно приводить к числовому виду

Нужно указать, какое число является "пропуском"

При сплите, алгоритм смотрит в какую сторону лучше отвести объекты с пропуском.

In [ ]:
xgtrain_missed = xgb.DMatrix(X[test_index], label=y[test_index], missing=-999.0)

### Feature importances

Подсчитывает сколько раз каждый признак использовался для использовался в вершине дерева при разбиении

Это не качество фактора, а его важность

In [ ]:
bst.get_fscore()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(12, 12))
xgb.plot_importance(bst)

###  Tree Plotting

Для наглядности можно отрисовать отдельное дерево. На ребрах также написано дефолтное направление для данных с пропусками

In [ ]:
_, ax = plt.subplots(figsize=(10, 10))
xgb.plot_tree(bst, num_trees=1, ax=ax)

### Прунинг

Обычно GBM перестает разделять вершины дерева, когда gain становится отрицательным - жадный подход.
Могло оказаться так, что после неудачного сплита с отрицательным gain'ом получится сделать сильно положительный сплит.

XGBoost доводит деревья до max_depth, после чего начинает удалять сплиты, которые несут отрицательный вклад.

### Дообучение модели

In [102]:
dtrain = xgb.DMatrix('/Users/klunev/xgboost/demo/data/agaricus.txt.train')
dtest = xgb.DMatrix('/Users/klunev/xgboost/demo/data/agaricus.txt.test')
watchlist  = [(dtest,'eval'), (dtrain,'train')]
###
# advanced: start from a initial base prediction
#
print ('start running example to start from a initial prediction')
# specify parameters via map, definition are same as c++ version
param = {'max_depth':2, 'eta':1, 'silent':1, 'objective':'binary:logistic' }
# train xgboost for 1 round
bst = xgb.train( param, dtrain, 1, watchlist )

# Note: we need the margin value instead of transformed prediction in set_base_margin
# do predict with output_margin=True, will always give you margin values before logistic transformation
ptrain = bst.predict(dtrain, output_margin=True)
ptest  = bst.predict(dtest, output_margin=True)

dtrain.set_base_margin(ptrain)
dtest.set_base_margin(ptest)

print ('this is result of running from initial prediction')
bst = xgb.train( param, dtrain, 1, watchlist )

XGBoostError: b'[19:06:51] src/io/local_filesys.cc:91: LocalFileSystem.ListDirectory /Users/klunev/xgboost/demo/data error: No such file or directory\n\nStack trace returned 10 entries:\n[bt] (0) 0   libxgboost.dylib                    0x0000000114e09826 dmlc::StackTrace() + 246\n[bt] (1) 1   libxgboost.dylib                    0x0000000114e095df dmlc::LogMessageFatal::~LogMessageFatal() + 47\n[bt] (2) 2   libxgboost.dylib                    0x0000000114eef568 dmlc::io::LocalFileSystem::ListDirectory(dmlc::io::URI const&, std::__1::vector<dmlc::io::FileInfo, std::__1::allocator<dmlc::io::FileInfo> >*) + 424\n[bt] (3) 3   libxgboost.dylib                    0x0000000114ed6780 dmlc::io::InputSplitBase::ConvertToURIs(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&) + 304\n[bt] (4) 4   libxgboost.dylib                    0x0000000114ed57a4 dmlc::io::InputSplitBase::InitInputFileInfo(std::__1::basic_string<char, std::__1::char_traits<char>, std::

**Hint**: в качестве начального приближения можно выбрать предсказания линейной модели

Обратите внимание на *watchlist* - множество выборок для которых мы считаем ошибку. Можно поставить, например, несколько валидационных сетов.
Примеры:
  - time-series prediction: каждая следующая валидационная выборка - новый месяц
  - ranking: ранжирование документов, хотим отслеживать качество на разных типах запросов (разные темы/разные языки и т.д)

### Встроенная кросс валидация

In [ ]:
xgb.cv(param, dtrain, nfold = 4, num_boost_round=15)


### Overfitting

  - regularization
  - subsampling
  - shrinkage
  - early stopping

<img src="https://i.imgur.com/mlFHAw2.png" height="600" width="600">


## Early stopping

 Параметры xgb.train, необходимые для early stopping
   - early_stopping_rounds
   - evals



### Веса для объектов

Мы можем учитывать каждый объект со своим весом, этот вес будет учитываться и при выборе бакетов при приближенном построении деревьев, при сплите, при подсчете Objective.

Допустим, мы хотим классифицировать короткие сообщения.  Некоторые из них повторяются. В этом случае выгодно "слить" вместе все дубликаты и посчитать их один раз, но с большим весом. При неизменном качестве это уменьшит время обучения 

In [ ]:
repeats = np.random.randint(low=1, high=5, size=X.shape[0])
train_examples = 300


X_train = X[:train_examples]
X_test = X[train_examples:]
y_train = y[:train_examples]
y_test = y[train_examples:]


X_train_repeated = np.repeat(X_train, repeats[:train_examples], axis=0)
X_test_repeated = np.repeat(X_test, repeats[train_examples:], axis=0)
y_train_repeated = np.repeat(y_train, repeats[:train_examples], axis=0)


xgtrain_repeated = xgb.DMatrix(X_train_repeated, label=y_train_repeated)
xgtrain_weighted = xgb.DMatrix(X_train, label=y_train, weight=repeats[:train_examples])

xgtest = xgb.DMatrix(X_test, label=y_test)

bst = xgb.train(params, xgtrain_repeated)
print "Repeated dataset. Train size: {}, error: {}".format(xgtrain_repeated.num_row(), bst.eval(xgtest))

bst = xgb.train(params, xgtrain_weighted)
print "Weighted dataset. Train size: {}, error: {}".format(xgtrain_weighted.num_row(), bst.eval(xgtest))


### 3 вида бустеров

* gbtree - обычные решающие деревья

* gblinear - линейные модели

* dart - решающие деревья, алгоритм может "выбрасывать" некоторые из деревьев, уменьшая переобучение


###  Алгоритм	Viola-Jones


Обнаружение объектов (обычно лиц) в реальном времени


<img src="http://farm8.staticflickr.com/7205/6884354620_49503b7c3a_z.jpg">

  - различных признаков очень много
  - используется вариация AdaBoost'a

# Другие параметры

### Бустинг

<i> learning_rates </i> - можно настроить убывающую скорость

### Параметры деревьев

<font size=3>
<i> max_depth </i> - максимальная глубина дерева. Слишком большая глубина ведет к переобучению

<i> subsample, colsample_bytree, colsample_bylevel </i> - сэмплирование по объектам и признакам


<i> min_child_weight </i> - минимальная сумма весов в листе

<i> scale_pos_weight </i> - вес целого класса, используется если один класс заметно чаще встречается, чем другой


### Дополнительные параметры для DART

<font size=3>
<i> sample_type </i> - стратегия выбора деревьев для выкидывания

<i> rate_drop </i> - какую долю выкидываем

<i> skip_drop </i> - шанс пропустить дроп на этой итерации

# Настраиваем XGBoost 

<font size=3>
* Выбираем относительно большую learning_rate ($ \eta \in [0.05, 0.3]$), подбираем оптимальное число деревьев для выбранного $ \eta $

* Настраиваем параметры деревьев, начиная с самых значимых (max_depth, min_child_weight, gamma, subsample, colsample_bytree)

* Настраиваем регуляризации ($ \lambda, \alpha $)

* Уменьшаем learning_rate, пропорционально увеличиваем число деревьев